### 1. Problem

**Problem:**

> The Excel file ***Capital_Budget.xlsx*** contains the annual budget, net profit for each project, and the associated costs broken down by project and by year.
Please help us to figure out which projects is supposed to select in order to maximize the total NPV. The values are in millions.

* Unused funds in one year can be carried over to the following year.
* Poject 2 cannot be selected unless Project 1 is also selected.
* Project 4 and 5 must not be selected together.
* At leaset one of projects 3, 4, and 5 must be selected.


In [48]:
import pandas as pd
df = pd.read_excel('Capital_Budget.xlsx', index_col=0)
df

,Project_1,Project_2,Project_3,Project_4,Project_5,Budget
Year_1,5,11,13,16,9,32.0
Year_2,2,8,7,7,6,30.0
Year_3,2,6,6,6,1,15.0
Year_4,0,3,4,6,-1,11.0
NPV,10,18,22,25,12,NaN


### 2. Define Variables, Objective Function, Constraints

Xi = 1 if project i is selected, otherwise 0   i = 1,2,3,4,5

Maximize Z = 10X1 + 18X2 + 22X3 + 25X4 + 12X5

S.T.

    5X1 + 11X2 + 13X3 + 16X4 +  9X5 <= 32   Year 1
    7X1 + 19X2 + 20X3 + 23X4 + 15X5 <= 62   Year 2
    9X1 + 25X2 + 26X3 + 29X4 + 16X5 <= 77   Year 3
    9X1 + 28X2 + 30X3 + 35X4 + 15X5 <= 88   Year 4
    X2 <= X1 ==> -X1 + X2 <=0    Project 2 cannot be selected, unless project 1 is also selected
                  X4 + X5 <= 1   Project 4 and 5 cannot be selected together
             X3 + X4 + X5 >= 1   At least one of projects 3,4,5 must be selected

### 3. Coding 

In [ ]:
import pandas as pd
import numpy as ny
import gurobipy as gp
from gurobipy import Model, quicksum, GRB

In [71]:
df_cost_budget = df.iloc[:-1]
df_cost_budget = df_cost_budget.cumsum()
df_cost_budget

,Project_1,Project_2,Project_3,Project_4,Project_5,Budget
Year_1,5,11,13,16,9,32.0
Year_2,7,19,20,23,15,62.0
Year_3,9,25,26,29,16,77.0
Year_4,9,28,30,35,15,88.0


In [72]:
var_names = df.columns.tolist()[:-1]
# ['Project_1', 'Project_2', 'Project_3', 'Project_4', 'Project_5']
year_names = df.index.values.tolist()[:-1]
# ['Year_1', 'Year_2', 'Year_3', 'Year_4']
constr_proj_select_names = ['constr_proj_select_1', 'constr_proj_select_2', 'constr_proj_selec3']

In [73]:
proj_n = len(var_names)
year_n = len(year_names)

In [74]:
df.loc['NPV', 'Project_1']

10

In [75]:
# create a new model
model = Model()

# set decision variables
X = model.addVars(var_names, vtype=GRB.BINARY, name='X')

# set the bojective function
model.setObjective(quicksum(df.loc['NPV', var]*X[var] for var in var_names), GRB.MAXIMIZE)

# Add constraints
# constraints on annual budget
for year in year_names:
    model.addConstr(
        quicksum(df_cost_budget.loc[year, var]*X[var] for var in var_names) <= df_cost_budget.loc[year, 'Budget']
        , name=year
    )
# constraints on combination of project selection 
model.addConstr(-1*X['Project_1'] + X['Project_2'] <= 0, name='Project_Selection_Combination_1')
model.addConstr(X['Project_4'] + X['Project_5'] <= 1, name='Project_Selection_Combination_2')
model.addConstr(X['Project_3'] + X['Project_4'] + X['Project_5'] >= 1, name='Project_Selection_Combination_3')

# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 19.6.0 19H2026)

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 5 columns and 27 nonzeros
Model fingerprint: 0x6c562792
Variable types: 0 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]
Found heuristic solution: objective 50.0000000
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 5 rows, 5 columns, 15 nonzeros
Variable types: 0 continuous, 5 integer (5 binary)
Found heuristic solution: objective 53.0000000

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.06 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 2: 53 50 

Optimal solution foun

In [ ]:
import numpy as np
import pandas as pd
from gurobipy import Model, GRB, quicksum

In [76]:
print(f"ObjVal = {model.ObjVal:.2f}")
model.printAttr('X')

ObjVal = 53.00

    Variable            X 
-------------------------
X[Project_1]            1 
X[Project_2]            1 
X[Project_4]            1 


In [79]:
if model.status == GRB.OPTIMAL:
    print("\n✅ Optimal solution found:\n")
    print(f'Objective value: {model.ObjVal}\n')
    
    #Print all variables and their optimal values
    for var in model.getVars():
        print(f"{var.VarName}: {var.X:.0f}")
else:
    Print('❌ No optimal solution found.')


✅ Optimal solution found:

Objective value: 53.0

X[Project_1]: 1
X[Project_2]: 1
X[Project_3]: 0
X[Project_4]: 1
X[Project_5]: 0


### Conclusion

* Based on the output of the linear programming model, the optimal project selection to maximize net profit includes:

    * Project 1
    * Project 2
    * Project 4

* This selection yields a total projected profit of 53 million.

* The decision was made under the defined budgetary and operational constraints, ensuring an efficient allocation of available resources to maximize fiancial return

**Problem:**

Company receives order to supply 1000 units

|                    | Machine_1 | Machine_2 | Machine_3 |
|--------------------|-----------|-----------|-----------|
| Variable Cost/unit |   2.39    |   1.99    |    2.99   | 
| Fixed Cost         |   300     |   250     |    400    | 
| Capacity           |   400     |   550     |    600    | 


Xi = number of units produced on machine i

Yi = 1 if machine i is used, 0 otherwise

In [37]:
import gurobipy as gp
from gurobipy import Model, GRB, quicksum

# Create a new model
model = gp.Model("MultiProductFixedCost")

# Set product from machines
machines = ['Machine_1', 'Machine_2', 'Machine_3']

# Set parameters
variable_cost = {'Machine_1': 2.39, 'Machine_2': 1.99, 'Machine_3': 2.99}
fixed_cost = {'Machine_1': 300, 'Machine_2': 250, 'Machine_3': 400}
capacity = {'Machine_1': 400, 'Machine_2': 550, 'Machine_3': 600}
order = 1000 # to supply 1000 units

# Set decision variables: INTEGER, CONTINUOUS, BINARY
X = model.addVars(machines,lb=0, vtype=GRB.CONTINUOUS, name='Prod')
Y = model.addVars(machines,lb=0, vtype=GRB.BINARY, name='Setup')

# Set objective function GRB.MAXIMIZE
model.setObjective(
    gp.quicksum(variable_cost[m]*X[m] + fixed_cost[m]*Y[m] for m in machines),
    GRB.MINIMIZE
)

# Add constraints
model.addConstr(gp.quicksum(X[m] for m in machines) == order, 'Order')
for m in machines:
    model.addConstr(X[m] <= capacity[m]*Y[m], f'{m}_Capacity')

# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 19.6.0 19H2026)

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 4 rows, 6 columns and 9 nonzeros
Model fingerprint: 0xcda464c6
Variable types: 3 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [2e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]
Presolve removed 4 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 1: 3090 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.090000000000e+03, best bound 3.090000000000e+03, gap 0.0000%


In [38]:
print(f"ObjVal = {model.ObjVal:.2f}")
model.printAttr('X')

ObjVal = 3090.00

    Variable            X 
-------------------------
Prod[Machine_2]          550 
Prod[Machine_3]          450 
Setup[Machine_2]            1 
Setup[Machine_3]            1 
